# Put simulation data in a usable form

In [1]:
import pylab as p
import numpy as np
from astropy.io import fits
import glob

# For saving images
import scipy.misc

In [2]:
data_dir = "../data/deep_lensing_originals/"
out_dir = "../data/deep_lensing_stamps/"

In [47]:
# Takes a fits filename (including filepath), and returns a set of cut up images
def get_images(file_name, nx_pixels=256, ny_pixels=256, include_overlaps=False):
    
    # Load first file
    snap1 = fits.open(file_name)
    scidata = snap1[0].data

    # Number of images that will fit in each image
    nx_images = len(scidata) / nx_pixels
    ny_images = len(scidata[0]) / ny_pixels
        
    # Total number of images
    n_images = nx_images * ny_images
    
    # Add to total the number of images gained from overlaps
    if include_overlaps:
        n_images += (nx_images - 1) * (ny_images - 1)

    # Create numpy array to hold images
    images = np.empty((n_images, nx_pixels, ny_pixels))

    # Create images
    for i in np.arange(nx_images):
        for j in np.arange(ny_images):
            index_image = i*nx_images + j
            images[index_image] = scidata[nx_pixels*i:nx_pixels*(i+1), ny_pixels*j:ny_pixels*(j+1)]
    
    # Add overlapping images
    if include_overlaps:
        for i in np.arange(nx_images-1):
            for j in np.arange(ny_images-1):
                index_image = nx_images*ny_images + i*(nx_images-1) + j
                images[index_image] = scidata[nx_pixels*(i+0.5):nx_pixels*(i+1.5), ny_pixels*(j+0.5):ny_pixels*(j+1.5)]
        
    return images

In [48]:
def degrade_shrink_image(images, bin_by):

    n_images = len(images[::])

    nx_pixels_in = len(images[0][0])
    ny_pixels_in = len(images[0].T[0])

    nx_pixels_out = nx_pixels_in / bin_by
    ny_pixels_out = ny_pixels_in / bin_by
    
    pictures = np.empty((n_images, nx_pixels_out, ny_pixels_out))

    
    for p in np.arange(n_images):

        vecs = np.zeros((nx_pixels_in, ny_pixels_out))
        for i in np.arange(nx_pixels_in):
            vecs[i] = 0.5 * (images[p][i][::2] + images[p][i][1::2])

        pictures[p] = 0.5 * (vecs[::2] + vecs[1::2])
    
        
    return pictures

In [49]:
def degrade_image(images, bin_by):

    n_images = len(images[::])

    nx_pixels = len(images[0][0])
    ny_pixels = len(images[0].T[0])
    
    pictures = np.empty((n_images, nx_pixels, ny_pixels))
    
    for p in np.arange(n_images):

        vecs = np.zeros((nx_pixels, ny_pixels))
        for i in np.arange(nx_pixels):
            vecs[i][::2] = 0.5 * (images[p][i][::2] + images[p][i][1::2])
            vecs[i][1::2] = vecs[i][::2]
            
        pictures[p][::2] = 0.5 * (vecs[::2] + vecs[1::2])
        pictures[p][1::2] = pictures[p][::2]
        
    return pictures

In [54]:
folder = "w_1.2/"
file_names = glob.glob(data_dir + folder + "*.fits")

i = 0
for f in file_names:
    images = get_images(f, include_overlaps=True)
    pic_out = degrade_image(images, 2)

    print f

    for j in np.arange(len(images[::])):
        i += 1
        scipy.misc.imsave(out_dir + folder + 'outfile_' + str(i) + '.png', pic_out[j])
        

../data/deep_lensing_originals/w_1.2/snap11_potentialPlane2_normal0.fits
../data/deep_lensing_originals/w_1.2/snap1_potentialPlane1_normal0.fits
../data/deep_lensing_originals/w_1.2/snap28_potentialPlane0_normal0.fits
../data/deep_lensing_originals/w_1.2/snap24_potentialPlane0_normal0.fits
../data/deep_lensing_originals/w_1.2/snap24_potentialPlane2_normal0.fits
../data/deep_lensing_originals/w_1.2/snap6_potentialPlane0_normal0.fits
../data/deep_lensing_originals/w_1.2/snap29_potentialPlane1_normal0.fits
../data/deep_lensing_originals/w_1.2/snap20_potentialPlane2_normal0.fits
../data/deep_lensing_originals/w_1.2/snap2_potentialPlane2_normal0.fits
../data/deep_lensing_originals/w_1.2/snap8_potentialPlane0_normal0.fits
../data/deep_lensing_originals/w_1.2/snap22_potentialPlane1_normal0.fits
../data/deep_lensing_originals/w_1.2/snap9_potentialPlane1_normal0.fits
../data/deep_lensing_originals/w_1.2/snap5_potentialPlane2_normal0.fits
../data/deep_lensing_originals/w_1.2/snap41_potentialPlan

/usr/local/lib/python2.7/dist-packages/ipykernel-4.2.2-py2.7.egg/ipykernel/__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
